In [ ]:
genome_regions = {}
entities = ['gene', 'exon', 'CDS']
#path = '../results/regions.tsv'
path = '../results/regions_chr21.tsv' # for testing

# genomic regions obtention
with open(path, 'r') as file:
    for line in file:
        chr, entity, start, stop = line.strip().split('\t')

        if chr not in genome_regions.keys():
            genome_regions[chr] = {entity_type: list() for entity_type in entities}
        genome_regions[chr][entity].append((int(start), int(stop)))

# regions preparation
for chr in genome_regions.keys():
    for entity in entities:

        # sort regions deleting duplicates
        genome_regions[chr][entity] = sorted(list(set(genome_regions[chr][entity])))
        
        # delete redundancies
        my_ranges = genome_regions[chr][entity]
        non_redundant_ranges = [my_ranges[0]]

        for i in range(len(my_ranges)-1):

            if my_ranges[i][0] - non_redundant_ranges[-1][1] > 0:
                non_redundant_ranges.append(my_ranges[i])
            else:
                new_range = (non_redundant_ranges[-1][0], my_ranges[i][1])
                non_redundant_ranges.pop()
                non_redundant_ranges.append(new_range)
            
        genome_regions[chr][entity] = [range(start, stop) for start, stop in non_redundant_ranges]

In [ ]:
for chr in genome_regions.keys():
    for entity in entities: 
        print(chr, entity, genome_regions[chr][entity])

In [ ]:
def is_in_ranges(x: int, rangesList: list) -> bool:
    for my_range in rangesList:
        if x in my_range: return True
    return False

    
def who_are_you(chr: str, pos: int) -> str:
    if is_in_ranges(pos, genome_regions[chr]['gene']):
        if is_in_ranges(pos, genome_regions[chr]['exon']):
            if is_in_ranges(pos, genome_regions[chr]['CDS']):
                return 'CDS'
            else: return 'exonic'
        else: return 'genic'
    else: return 'intergenic'

In [ ]:
geninc = 5012000
intergenic = 5020000
exonic = 5011850
intronic = 5012000
CDS = 10538729
not_CDS = 10538735

elements = [geninc, intergenic, exonic, intronic, CDS, not_CDS]
elements = [('chr21', element) for element in elements]

for element in elements:
    chr = element[0]
    pos = element[1]
    print(who_are_you(chr, pos))

In [ ]:
#extract positions from clinvar by chromosome

path = "results/clinvar_clean.tsv"

clinvar_positions = {}
with open(path, "r") as file:
    for line in file:
        chromosome, position, *_ = line.strip().split("\t")
        
        if chromosome not in clinvar_positions.keys():
            clinvar_positions[chromosome] = [int(position)]
        else: clinvar_positions[chromosome].append(int(position))
